In [1]:
#using python 3.8.10
#imports 
import pandas as pd
from datetime import datetime
import csv
import numpy as np

#here we are reading in our criminal data
PII_hearttable = pd.read_csv('data/dev_sample_HeartTable_10_17_2022.csv')
acxioim_df = pd.read_csv("data/out/acxiom_demo_cleaned_v1.csv")
agr_df = pd.read_csv("data/out/agr_demo_cleaned_v1.csv")
appriss_df = pd.read_csv("data/out/appriss_crime_cleaned_v1.csv")

In [2]:
# if the coulumn 'source_id' contains the substring "agr" then the class is 1 else 2
PII_hearttable['is_suicide'] = np.where(PII_hearttable['source_id'].str.contains('agr'), 0, 1)

# cut down PII_hearttable to only the columns we need
PII_hearttable = PII_hearttable[['source_id', 'is_suicide']]


In [3]:
# print all the columns with the same 'source_id'
appriss_df[appriss_df.duplicated(subset=['source_id'], keep=False)]

,source_id,999,1099,1199,1299,1399,2099,3806,5299,5401,...,7399_HasCrime,tier_1_HasCrime,tier_2_HasCrime,tier_3_HasCrime,tier_4_HasCrime,CDCcat_A_HasCrime,CDCcat_B_HasCrime,CDCcat_C_HasCrime,CDCcat_D_HasCrime,CDCcat_E_HasCrime


In [4]:
new_columns = {}
for col in appriss_df.columns:
    if col != 'source_id':
        new_columns[col] = col + '_appriss'

# use the rename method to rename the columns
appriss_df = appriss_df.rename(columns=new_columns)

# combine acxioim_df and PII_hearttable on column 'source_id'
data_to_model_v1 = pd.merge(PII_hearttable, appriss_df, on='source_id', how='left')

data_to_model_v1 = data_to_model_v1.fillna(0)

In [5]:
# print all the columns with the same 'source_id'
acxioim_df[acxioim_df.duplicated(subset=['source_id'], keep=False)]

,source_id,Single Parent,Senior Adult in Household,Young Adult in Household,Education - 1st Person in Household - 100%,Guns and Ammunition,Hunting,Underbanked,Economic Stability Indicator,Population Density,...,Generations in Household_2.0,Generations in Household_3.0,Marital Status in the Household - 100%_A,Marital Status in the Household - 100%_B,Marital Status in the Household - 100%_M,Marital Status in the Household - 100%_S,Presence of Children - 100%_N,Presence of Children - 100%_Y,Home Owner / Renter - 100%_O,Home Owner / Renter - 100%_R
19481,4407_(medical_source)index_2310,0.0,0,0,1.0,0.0,0.0,1,9,1.0,...,0,0,0,1,0,0,1,0,1,0
19482,4407_(medical_source)index_2310,0.0,0,0,1.0,0.0,0.0,1,9,1.0,...,0,0,0,1,0,0,1,0,1,0
19483,4408_(medical_source)index_6634,0.0,0,0,1.0,0.0,0.0,1,9,1.0,...,0,0,0,1,0,0,1,0,1,0
19484,4408_(medical_source)index_6634,0.0,0,0,1.0,0.0,0.0,1,9,1.0,...,0,0,0,1,0,0,1,0,1,0


In [6]:
acxioim_df = acxioim_df.drop_duplicates(subset=['source_id'], keep='first')

# to all the columns in acxioim_df besides 'source_id' add to them '_acxiom' in the column name
# create a dictionary to map the old column names to the new ones
new_columns = {}
for col in acxioim_df.columns:
    if col != 'source_id':
        new_columns[col] = col + '_acxiom'

# use the rename method to rename the columns
acxioim_df = acxioim_df.rename(columns=new_columns)

# combine acxioim_df and PII_hearttable on column 'source_id'
data_to_model_v1 = pd.merge(data_to_model_v1, acxioim_df, on='source_id', how='left')

In [7]:
# print all the columns with the same 'source_id'
agr_df[agr_df.duplicated(subset=['source_id'], keep=False)]

,source_id,VACANT,NO STAT,RESIDENTIAL,DEMO_VETERAN PRESENT IN HH,MULTIPLE PROPERTIES COUNT,BANKRUPCY,DUI,SEX OFFENDER,DIGITAL FLAG,...,OWNER OCCUPIED STATUS_N,OWNER OCCUPIED STATUS_R,OWNER OCCUPIED STATUS_Y,OWNER STATUS TYPE_I,OWNER STATUS TYPE_O,OWNER STATUS TYPE_P,OWNER STATUS TYPE_T,PROPERTY TYPE DESC_A,PROPERTY TYPE DESC_C,PROPERTY TYPE DESC_R
8951,4407_(medical_source)index_2310,0.0,1.0,1,0.0,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
8952,4408_(medical_source)index_6634,0.0,1.0,1,0.0,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
8954,4407_(medical_source)index_2310,0.0,1.0,1,0.0,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
8955,4408_(medical_source)index_6634,0.0,1.0,1,0.0,NaN,0.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [8]:
agr_df = agr_df.drop_duplicates(subset=['source_id'], keep='first')
new_columns = {}
for col in agr_df.columns:
    if col != 'source_id':
        new_columns[col] = col + '_agr'

# use the rename method to rename the columns
agr_df = agr_df.rename(columns=new_columns)

# combine acxioim_df and PII_hearttable on column 'source_id'
data_to_model_v1 = pd.merge(data_to_model_v1, agr_df, on='source_id', how='left')


In [9]:
# write the data to a csv file
data_to_model_v1.to_csv('data/out/model_data_v1.csv', index=False)